<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9cedea7b7200476fe56db3028990e1369c96ae931f7cd98cc3466c4a2671ff26
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 09:24:38
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 53.80 K (0.38%)
Current PnL: -19.97 L (-13.3%)
CY Booked + Current PnL: -8.53 L (-5.68%)
-------------------
Total profit:  1.76 L
Total loss:  -21.73 L
-------------------
Total Booked + Current PnL: 18.45 L (14.99%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.08%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.47 L (61.77%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,67.0,H-MC,14.32,119897.0,19393.0,9712.0,-1.45,19.30,8.10,28.96,79.0,2.00,0.85,59.82,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.32,54.0,X-MC,3.20,192170.0,27500.0,15835.0,1.55,16.70,8.24,26.31,89.0,1.74,1.36,26.02,X40N,NTT,AC
77,TTKPRESTIG,770.00,94.45,42.0,M-SC,0.98,83742.0,-17035.0,17125.0,0.35,-16.90,20.45,0.09,245.0,-0.99,0.59,10.02,OX40N,NTT,DURABLES
84,VOLTAS,1530.00,0.88,60.0,X-MC,1.08,211785.0,20043.0,17705.0,2.16,10.45,8.36,19.69,99.0,1.13,1.50,17.96,XY25,NTT,AC
19,CIPLA,1662.74,-22.52,45.0,X-LC,3.61,207089.0,2589.0,20709.0,-0.10,1.27,10.00,11.39,10.0,0.13,1.47,10.33,X40N,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,122.36,54.0,H-SC,7.51,150550.0,-67946.0,96141.0,4.82,-31.10,63.86,12.90,153.0,-0.71,1.07,14.60,OX40N,NTT,FINANCE
85,WHIRLPOOL,2270.00,-47.23,37.0,M-SC,2.37,89212.0,-2286.0,81040.0,3.97,-2.50,90.84,86.07,223.0,-0.03,0.63,29.55,XR,NTT,DURABLES
60,SAIL,228.00,49.16,61.0,M-MC,12.67,234984.0,10022.0,157627.0,3.60,4.45,67.08,74.53,192.0,0.06,1.67,38.15,XY24,BTT,STEEL
55,RAJOOENG,143.10,-37.60,54.0,H-SC,17.12,94450.0,-8050.0,48651.0,2.90,-7.85,51.51,39.61,114.0,-0.17,0.67,15.35,AR,ATH,MISC
46,KPIGREEN,731.05,-8.64,34.0,H-SC,6.29,108864.0,-16433.0,75356.0,2.42,-13.12,69.22,47.03,141.0,-0.22,0.77,36.31,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,61.0,M-LC,7.74,214455.0,9379.0,103196.0,-3.76,4.57,48.12,54.89,52.0,0.09,1.52,29.77,X5K,ATH,METALS
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.00,80640.0,-32909.0,72963.0,-1.64,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
50,NATIONALUM,244.55,-39.52,67.0,H-MC,14.32,119897.0,19393.0,9712.0,-1.45,19.30,8.10,28.96,79.0,2.00,0.85,59.82,MH,ATH,METALS
74,TCS,4406.34,-28.36,51.0,X-LC,12.15,285393.0,-60563.0,128798.0,-0.84,-17.51,45.13,19.73,1.0,-0.47,2.02,5.11,X40,ATH,IT
18,CERA,9475.00,-24.58,33.0,H-SC,1.81,137540.0,-38363.0,80378.0,-0.71,-21.81,58.44,23.89,149.0,-0.48,0.98,18.78,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.53,130167.0,-12312.0,31839.0,1.07,-8.64,24.46,13.70,163.0,-0.39,0.92,53.63,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.45,42.0,M-SC,0.98,83742.0,-17035.0,17125.0,0.35,-16.90,20.45,0.09,245.0,-0.99,0.59,10.02,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,50.0,H-SC,1.94,221841.0,-47826.0,84166.0,0.46,-17.74,37.94,13.47,138.0,-0.57,1.57,12.80,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.03,53.0,H-MC,6.54,105384.0,-26451.0,68679.0,-0.48,-20.06,65.17,32.03,98.0,-0.39,0.75,18.04,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137540.0,-38363.0,80378.0,-0.71,-21.81,58.44,23.89,149.0,-0.48,0.98,18.78,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,61.0,M-MC,12.67,234984.0,10022.0,157627.0,3.60,4.45,67.08,74.53,192.0,0.06,1.67,38.15,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,61.0,M-LC,7.74,214455.0,9379.0,103196.0,-3.76,4.57,48.12,54.89,52.0,0.09,1.52,29.77,X5K,ATH,METALS
86,WIPRO,420.00,-12.43,54.0,M-LC,5.73,153834.0,2889.0,106561.0,0.70,1.91,69.27,72.51,53.0,0.03,1.09,8.05,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,54.0,M-LC,9.12,156667.0,15827.0,28670.0,-0.03,11.24,18.30,31.59,66.0,0.55,1.11,40.70,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,11.91,193757.0,13949.0,98467.0,0.15,7.76,50.82,62.52,88.0,0.14,1.37,44.49,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,67.0,H-MC,14.32,119897.0,19393.0,9712.0,-1.45,19.30,8.10,28.96,79.0,2.00,0.85,59.82,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,11.91,193757.0,13949.0,98467.0,0.15,7.76,50.82,62.52,88.0,0.14,1.37,44.49,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,61.0,M-LC,7.74,214455.0,9379.0,103196.0,-3.76,4.57,48.12,54.89,52.0,0.09,1.52,29.77,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.45,43.0,H-SC,8.74,127258.0,3922.0,127703.0,-0.46,3.18,100.35,106.72,119.0,0.03,0.90,27.30,AR,ATH,MISC
86,WIPRO,420.00,-12.43,54.0,M-LC,5.73,153834.0,2889.0,106561.0,0.70,1.91,69.27,72.51,53.0,0.03,1.09,8.05,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.11,30.0,X-SC,37.81,52637.0,-12369.0,163301.0,0.99,-19.03,310.24,232.18,198.0,-0.08,0.37,0.99,XY24,NTT,MISC
23,DABUR,735.00,-12.30,35.0,X-MC,3.75,237141.0,-10927.0,119329.0,0.73,-4.40,50.32,43.70,102.0,-0.09,1.68,9.40,XY24,BTT,FMCG
22,COLPAL,3767.14,-5.20,36.0,X-MC,7.29,221460.0,-41905.0,155243.0,0.27,-15.91,70.10,43.04,84.0,-0.27,1.57,2.81,XY25,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,36.0,X-LC,5.55,206756.0,-45012.0,98230.0,0.56,-17.88,47.51,21.13,27.0,-0.46,1.47,11.21,X40,ATH,PAINTS
29,HAVELLS,2069.16,-6.32,36.0,X-MC,0.59,232300.0,-15431.0,90481.0,0.22,-6.23,38.95,30.30,92.0,-0.17,1.65,6.23,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-6.32,36.0,X-MC,0.59,232300.0,-15431.0,90481.0,0.22,-6.23,38.95,30.30,92.0,-0.17,1.65,6.23,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,0.88,60.0,X-MC,1.08,211785.0,20043.0,17705.0,2.16,10.45,8.36,19.69,99.0,1.13,1.50,17.96,XY25,NTT,AC
73,TATAMOTORS,1065.00,-21.50,44.0,X-LC,1.60,247421.0,-29102.0,139174.0,0.07,-10.52,56.25,39.81,54.0,-0.21,1.75,18.58,XY24,NTT,AUTO
65,SIEMENS,4671.50,-0.68,51.0,H-LC,1.70,159740.0,-26355.0,73832.0,0.52,-14.16,46.22,25.51,15.0,-0.36,1.13,17.72,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.49,51.0,X-MC,1.84,140378.0,4245.0,26882.0,0.76,3.12,19.15,22.86,91.0,0.16,1.00,16.12,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.11,30.0,X-SC,37.81,52637.0,-12369.0,163301.0,0.99,-19.03,310.24,232.18,198.0,-0.08,0.37,0.99,XY24,NTT,MISC
83,VBL,671.64,-21.10,40.0,X-LC,10.35,284133.0,-31709.0,144368.0,-0.12,-10.04,50.81,35.67,5.0,-0.22,2.01,2.42,X40N,ATH,FMCG
22,COLPAL,3767.14,-5.20,36.0,X-MC,7.29,221460.0,-41905.0,155243.0,0.27,-15.91,70.10,43.04,84.0,-0.27,1.57,2.81,XY25,ATH,FMCG
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196847.0,-3291.0,24173.0,0.66,-1.64,12.28,10.44,4.0,-0.14,1.40,3.83,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,40.0,X-MC,4.95,180744.0,-19762.0,116923.0,0.11,-9.86,64.69,48.45,90.0,-0.17,1.28,4.23,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.11,30.0,X-SC,37.81,52637.0,-12369.0,163301.0,0.99,-19.03,310.24,232.18,198.0,-0.08,0.37,0.99,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81977.0,-63183.0,139107.0,-0.42,-43.53,169.69,52.31,136.0,-0.45,0.58,9.73,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.58,45.0,X-SC,4.41,97724.0,-30946.0,76244.0,0.43,-24.05,78.02,35.20,219.0,-0.41,0.69,12.17,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.50,52.0,X-SC,1.44,108584.0,6580.0,37342.0,1.22,6.45,34.39,43.06,122.0,0.18,0.77,24.67,X40N,ATH,MISC
33,HONAUT,58357.33,-22.51,48.0,X-SC,2.13,109170.0,-18768.0,65906.0,0.41,-14.67,60.37,36.84,143.0,-0.28,0.77,11.76,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,51.0,X-LC,12.15,285393.0,-60563.0,128798.0,-0.84,-17.51,45.13,19.73,1.0,-0.47,2.02,5.11,X40,ATH,IT
40,INFY,2275.00,-17.09,58.0,X-LC,7.77,323483.0,10595.0,161095.0,0.62,3.39,49.80,54.87,3.0,0.07,2.29,10.26,X40,BTT,IT
42,ITC,452.00,-39.19,46.0,X-LC,2.40,196847.0,-3291.0,24173.0,0.66,-1.64,12.28,10.44,4.0,-0.14,1.40,3.83,X40,NTT,FMCG
83,VBL,671.64,-21.10,40.0,X-LC,10.35,284133.0,-31709.0,144368.0,-0.12,-10.04,50.81,35.67,5.0,-0.22,2.01,2.42,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,48.0,H-LC,7.49,249432.0,-12187.0,131401.0,0.97,-4.66,52.68,45.57,7.0,-0.09,1.77,6.14,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.00,80640.0,-32909.0,72963.0,-1.64,-28.98,90.48,35.27,268.0,-0.45,0.57,97.37,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,41.0,H-SC,10.35,90480.0,-7500.0,29107.0,0.13,-7.65,32.17,22.05,152.0,-0.26,0.64,31.52,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,67.0,H-MC,14.32,119897.0,19393.0,9712.0,-1.45,19.30,8.10,28.96,79.0,2.00,0.85,59.82,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.53,130167.0,-12312.0,31839.0,1.07,-8.64,24.46,13.70,163.0,-0.39,0.92,53.63,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,15.32,144720.0,-4500.0,148685.0,-0.16,-3.02,102.74,96.62,208.0,-0.03,1.03,56.62,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,65.0,H-SC,8.98,151807.0,-27264.0,92997.0,0.16,-15.23,61.26,36.71,135.0,-0.29,1.08,50.62,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,15.32,144720.0,-4500.0,148685.0,-0.16,-3.02,102.74,96.62,208.0,-0.03,1.03,56.62,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.71,61.0,M-LC,7.74,214455.0,9379.0,103196.0,-3.76,4.57,48.12,54.89,52.0,0.09,1.52,29.77,X5K,ATH,METALS
26,GREENPANEL,537.00,232.69,59.0,M-SC,4.67,148474.0,-34604.0,113583.0,-0.26,-18.90,76.50,43.14,230.0,-0.30,1.05,39.66,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,67.0,H-MC,14.32,119897.0,19393.0,9712.0,-1.45,19.30,8.10,28.96,79.0,2.00,0.85,59.82,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.12
1,25,43.43
2,50,75.13


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
LC    32.23
MC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.91
X40      14.64
X40N     11.20
XR       10.95
XY25     10.39
AR        8.20
OX40N     7.69
X200      1.74
MH        1.62
X5K       1.52
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.54
X-LC    22.86
X-MC    17.17
M-SC    12.51
H-LC     4.64
H-MC     4.62
X-SC     4.51
M-LC     3.72
M-MC     1.67
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.72,-5.93,40.84
IT,12.88,-15.75,78.44
FINANCE,10.54,-13.53,68.35
MISC,6.91,-17.84,83.14
BANKS,6.34,-11.54,70.22
PAINTS,5.50,-19.12,36.87
ELECTRICAL,5.32,-9.69,48.17
INSURANCE,3.80,-3.23,41.82
AC,3.55,3.38,12.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213168.0,22
XR,1364418.0,14
AR,1172968.0,9
X40,774970.0,10
X40N,653091.0,9
OX40N,563269.0,10
XY25,471055.0,7
SR,265770.0,2
X5K,103196.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3399439.0,25
M-SC,1277469.0,15
X-MC,1111742.0,12
X-LC,1014434.0,13
X-SC,542562.0,6
H-MC,358309.0,4
H-LC,285704.0,3
L-SC,260393.0,3
M-LC,238427.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196525.0      6
           AR         803447.0      5
           XR         798024.0      7
M-SC       XY24       689147.0      6
X-MC       XY24       589457.0      4
X-LC       X40        482147.0      5
X-SC       X40N       303017.0      4
H-SC       SR         265770.0      2
           OX40N      260317.0      4
X-LC       X40N       217316.0      3
X-MC       X40        216579.0      4
X-LC       XY24       206990.0      2
H-LC       AR         205233.0      2
H-MC       XY24       181451.0      1
X-MC       XY25       172948.0      2
L-SC       XR         171697.0      2
M-SC       AR         164288.0      2
X-SC       XY24       163301.0      1
M-MC       XY24       157627.0      1
M-SC       XY25       148685.0      1
           OX40N      145577.0      4
X-MC       X40N       132758.0      2
M-SC       XR         129772.0      2
X-LC       XY25       107981.0      3
M-LC       XR         106561.0      1
           X5K        103196.0      1
H-MC       XR          98467.0      1
L-SC       OX40N       88696.0      1
H-LC       X200        80471.0      1
X-SC       X40         76244.0      1
H-SC       MH          75356.0      1
H-MC       OX40N       68679.0      1
L-MC       XR          59897.0      1
L-LC       XY25        41441.0      1
M-LC       XY24        28670.0      1
H-MC       MH           9712.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
